# imports

In [2]:
from noises import *
from phantom import *

import torch
import torch.nn.functional as F

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as compare_psnr


import pandas as pd
import numpy as np

from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from noises import *
from phantom import *

# suppress warnings
import warnings
warnings.filterwarnings("ignore")



NOISE_FUNCTIONS = {
    'gaussian': {'function': add_gaussian_noise, 'param': 'noise_factor'},
    'speckle': {'function': add_speckle_noise, 'param': 'noise_factor'},
    'uniform': {'function': add_uniform_noise, 'param': 'noise_factor'},
    'exponential': {'function': add_exponential_noise, 'param': 'noise_factor'},
    'rayleigh': {'function': add_rayleigh_noise, 'param': 'noise_factor'},
    'erlang': {'function': add_erlang_noise, 'param': 'noise_factor'},  # You can also add 'scale' depending on what you want to vary
    'brownian': {'function': add_brownian_noise, 'param': 'noise_factor'},
    'stripe': {'function': add_stripe_noise, 'param': 'noise_factor'},
    'multiplicative': {'function': add_multiplicative_noise, 'param': 'noise_factor'}
}

# create folders

In [ ]:
# make directories
for key in NOISE_FUNCTIONS.keys():
    os.makedirs(f'./{key}', exist_ok=True)

In [ ]:
# make directories in the ground truth folder for each resolution
# 64, 128, 256, 512
for res in [64, 128, 256, 512]:
    os.makedirs(f'./ground_truth/{res}', exist_ok=True)


# generate ground truth and sample

In [ ]:
# now generate 50 phantoms for each resolution
# save the phantoms to the ground truth folder
for res in [6, 7, 8, 9]:
    for i in range(50):
        # res is 2**res
        # so a 64x64 image has a res of 6
        phantom = generate_phantom(res)
        # save the phantom
        # output is a np array
        np.save(f'./ground_truth/{2**res}/{i}.npy',phantom)

In [ ]:
# test plotting one of the phantoms

for res in [6, 7, 8, 9]:
    plt.figure()
    plt.imshow(np.load(f'./ground_truth/{2**res}/0.npy').squeeze(), cmap='gray')

# test noise

In [ ]:
# test applying each noise function to one of the phantoms
# the noise function works like this: add_selected_noise(img, noise_type='gaussian', **kwargs):

img = np.load(f'./ground_truth/64/0.npy').squeeze()
img_torch = phantom_to_torch(img)

for key in NOISE_FUNCTIONS.keys():
    plt.figure()
    noise_img = add_selected_noise(img_torch, noise_type=key)
    # include label of the noise type
    plt.title(key)
    plt.imshow(noise_img.squeeze(), cmap='gray', )

# not all noise is equal

In [3]:
### this first finds the average PSNR that corresponds to gaussian noise for leves [.05, .09, .15, .20]
### then it uses this average PSNR to target the input parameter for each noise function
### the output is a csv file with the noise type, resolution, noise level, noise factor, and PSNR

### this is a hacky way to approximate "equivalent noise levels" for each noise type

NOISE_FUNCTIONS = {
    'gaussian': {'function': add_gaussian_noise, 'param': 'noise_factor'},
    'speckle': {'function': add_speckle_noise, 'param': 'noise_factor'},
    'uniform': {'function': add_uniform_noise, 'param': 'noise_factor'},
    'exponential': {'function': add_exponential_noise, 'param': 'noise_factor'},
    'rayleigh': {'function': add_rayleigh_noise, 'param': 'noise_factor'},
    'erlang': {'function': add_erlang_noise, 'param': 'noise_factor'},  # You can also add 'scale' depending on what you want to vary
    'brownian': {'function': add_brownian_noise, 'param': 'noise_factor'},
    'stripe': {'function': add_stripe_noise, 'param': 'noise_factor'},
    'multiplicative': {'function': add_multiplicative_noise, 'param': 'noise_factor'}
}

def find_noise_factor(target_psnr, img_torch, noise_type):
    candidate_factors = np.linspace(0.001, 1, 10000)  # You can customize this range and granularity
    for noise_factor in candidate_factors:
        e_noise_img = add_selected_noise(img_torch, noise_type=noise_type, noise_factor=noise_factor)

        if np.isclose(compare_psnr(img_torch.numpy(), e_noise_img.numpy()), target_psnr, atol=0.1):  # tolerance level
            return noise_factor
    return None
    
def find_and_save_noise_factors(
        NOISE_FUNCTIONS,
        resolutions=[64, 128], 
        gaussian_levels=[0.05, 0.09, 0.15, 0.20], 
        num_gaussian_samples=100, 
        num_samples = 1000, 
        ):
    rows = []

    for resolution in resolutions:
        print(f'Starting resolution: {resolution}')
        csv_file_name= f'noise_analysis_{resolution}res.csv'
        img = np.load(f'./ground_truth/{resolution}/0.npy').squeeze()
        img_torch = phantom_to_torch(img)

        for gaussian_level in gaussian_levels:
            print(f'\tStarting Gaussian Level: {gaussian_level}')
            target_psnr_sum = 0
            num_gaussian_samples = 10  # Number of Gaussian noise samples to average over

            for _ in range(num_gaussian_samples):
                g_noise_img = add_selected_noise(img_torch, noise_type='gaussian', noise_factor=gaussian_level)
                target_psnr_sum += compare_psnr(img_torch.numpy(), g_noise_img.numpy())

            avg_target_psnr = target_psnr_sum / num_gaussian_samples

            for noise_type in NOISE_FUNCTIONS.keys():
                print(f'\t\tStarting Noise Type: {noise_type}')
                noise_factor_sum = 0  # Initialize sum
                num_samples = 1000  # Number of samples for other noises

                for _ in range(num_samples):
                    factor = find_noise_factor(avg_target_psnr, img_torch, noise_type)
                    noise_factor_sum += factor if factor is not None else 0
                
                avg_noise_factor = noise_factor_sum / num_samples if num_samples != 0 else 0
                
                # Save row
                rows.append([noise_type, resolution, gaussian_level, avg_noise_factor, "NA", avg_target_psnr])

                # Create DataFrame and save to CSV
                df = pd.DataFrame(rows, columns=["Noise_Type", "Resolution", "NL", "NF1", "NF2", "PSNR"])
                df.to_csv(csv_file_name, index=False)



In [4]:
# exectute the function
# this one can take a long time so partion or run somewhere else
find_and_save_noise_factors(
    NOISE_FUNCTIONS, 
    resolutions=[128],
    num_gaussian_samples=10, 
    num_samples = 100, 
    )

Starting resolution: 128
	Starting Gaussian Level: 0.05
		Starting Noise Type: gaussian
		Starting Noise Type: speckle
		Starting Noise Type: uniform
		Starting Noise Type: exponential
		Starting Noise Type: rayleigh


# check results

In [ ]:
resolution = 128
noise_df = pd.read_csv(f'noise_analysis_{resolution}res.csv')

for key in NOISE_FUNCTIONS.keys():
    print(f'\n----------------------------------\nNoise Type: {key}\n')
    print(f"\t{'Resolution':<12}{'NL':<10}{'NF':<10}{'PSNR':<10}{'Target PSNR':<15}{'PSNR Error':<15}")
    for res in [6, 7]:
        resolution = 2**res
        for noise_level in [0.05, 0.09, 0.15, 0.20]:
            # Load the phantom
            img = np.load(f'./ground_truth/{2**res}/0.npy').squeeze()
            img_torch = phantom_to_torch(img)
            
            target_psnr = noise_df[
                (noise_df['Noise_Type'] == key) &
                (noise_df['Resolution'] == resolution) &
                (noise_df['NL'] == noise_level)
            ]['PSNR'].values[0]
            
            noise_factor = noise_df[
                (noise_df['Noise_Type'] == key) &
                (noise_df['Resolution'] == resolution) &
                (noise_df['NL'] == noise_level)
            ]['NF1'].values[0]
                
            # Add noise to the image
            noise_img = add_selected_noise(img_torch, noise_type=key, noise_factor=noise_factor)
            
            # Calculate the psnr
            psnr = round(compare_psnr(img_torch.numpy(), noise_img.numpy()),3)

            psnr_error = round(abs(psnr-target_psnr), 3)
            print(f"\t{resolution:<12}{noise_level:<10}{noise_factor:<10}{psnr:<10.4f}{target_psnr:<15.4f}{psnr_error:<15.3f}")

In [ ]:
# check with plots
# resolution = 128
noise_df = pd.read_csv(f'noise_analysis_{resolution}res.csv')

for key in NOISE_FUNCTIONS.keys():
    for res in [6]:
        resolution = 2**res
        for noise_level in [0.05, 0.09, 0.15, 0.20]:
            # Load the phantom
            img = np.load(f'./ground_truth/{2**res}/0.npy').squeeze()
            img_torch = phantom_to_torch(img)
            
            target_psnr = noise_df[
                (noise_df['Noise_Type'] == key) &
                (noise_df['Resolution'] == resolution) &
                (noise_df['NL'] == noise_level)
            ]['PSNR'].values[0]
            
            noise_factor = noise_df[
                (noise_df['Noise_Type'] == key) &
                (noise_df['Resolution'] == resolution) &
                (noise_df['NL'] == noise_level)
            ]['NF1'].values[0]

            # if key == "brownian":
            #     noise_factor *= 1.5

            # if key == "stripe":
            #     noise_factor *= 1.5

            # Add noise to the image
            noise_img = add_selected_noise(img_torch, noise_type=key, noise_factor=noise_factor)

            # calculate the psnr
            psnr = round(compare_psnr(img_torch.numpy(), noise_img.numpy()),4)
            print(f'PSNR: {psnr} -- Target PSNR: {round(target_psnr,4)} -- Noise Level: {round(noise_level,4)} -- Noise Factor: {round(noise_factor,4)}')

            # plot noise image and clean image side by side
            plt.figure()
            plt.subplot(1, 2, 1)
            plt.imshow(img, cmap='gray')
            plt.title('Original')
            plt.subplot(1, 2, 2)
            plt.imshow(noise_img.squeeze(), cmap='gray')
            plt.title(f'{key.capitalize()} Noise')
            # wait a few seconds before move on to allow time to view the image
            plt.show(block=False)
            # plt.imshow(img, cmap='gray')


# now create noise level and resolution subfolders in the sample folder

In [ ]:
# first resolution subfolders 64, 128, 256, 512
# next noise level subfolders 0.05, 0.09, .15, .20

# these are subfolders of the noise type folders, which are subfolders of the current directory
# so the path is ./noise_type/resolution/noise_level/phantom_phantomid_noisetype_noiselevel.npy
for key in NOISE_FUNCTIONS.keys():
    for res in [6, 7, 8, 9]:
        for noise_level in [0.05, 0.09, 0.15, 0.20]:
            # only create the folder if it doesn't exist
            os.makedirs(f'./{key}/resolution_{2**res}/noise_level_{noise_level}', exist_ok=True)

# now apply these noise levels to the ground truth

In [ ]:
# finally we create the noisy phantoms

resolution = 128

noise_df = pd.read_csv(f'noise_analysis_{resolution}res.csv')
for key in NOISE_FUNCTIONS.keys():
    for img_id in range(50):
        img_np = np.load(f'./ground_truth/64/{img_id}.npy').squeeze()
        img_torch = phantom_to_torch(img_np)
        for noise_level in [0.05, 0.09, 0.15, 0.20]:
            noise_factor = noise_df[
                (noise_df['Noise_Type'] == key) &
                (noise_df['Resolution'] == 64) &
                (noise_df['NL'] == noise_level)
            ]['NF1'].values[0]
            noise_img = add_selected_noise(img_torch, noise_type=key, noise_factor=noise_factor)
            # save the image
            np.save(f'./{key}/res_64/nl_{noise_level}/p_{img_id}.npy', noise_img.numpy())
            print(f'Completed {key}-{img_id} noise with noise level {noise_level}')